In [1]:
import sys
sys.path.append('../upd')

import numpy as np

from Hamiltonian import HamiltonianAPI
from Huckel_Hamiltonian import *

Initializing dense array

In [2]:
norb = 2
nelec = norb
hubbard = Hubbard([(f"C{i}",f"C{i+1}", 1) for i in range(1,norb)], alpha=0, beta=-1,
                  u_onsite=np.array([1 for i in range(norb)]))
ecore, h, v = hubbard.get_hamilton()

2 8
1
7 13
1


/opt/conda/lib/python3.7/site-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
/opt/conda/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [8]:
from typing import TextIO
def save_fcidump(f: TextIO, core_energy, one_ints, two_ints, nelec=0, spinpol=0):
        """
        Save all parts of hamiltonian in fcidump format
        Adapted from https://github.com/theochem/iodata/blob/master/iodata/formats/fcidump.py
        :param f: TextIO file
        :param nelec: The number of electrons in the system
        :param spinpol: The spin polarization. By default, its value is derived from the
                        molecular orbitals (mo attribute), as abs(nalpha - nbeta). In this case,
                        spinpol cannot be set. When no molecular orbitals are present, this
                        attribute can be set.
        :return: None
        """
        # Write header
        nactive = one_ints.shape[0]
        print(f' &FCI NORB={nactive:d},NELEC={nelec:d},MS2={spinpol:d},', file=f)
        print(f"  ORBSYM= {','.join('1' for v in range(nactive))},", file=f)
        print('  ISYM=1', file=f)
        print(' &END', file=f)

        # Write integrals and core energy
        #two_ints = self.two_ints
        #for i in range(nactive):  # pylint: disable=too-many-nested-blocks
        #    for j in range(i + 1):
        #        for k in range(nactive):
        #            for l in range(k + 1):
        #                if (i * (i + 1)) / 2 + j >= (k * (k + 1)) / 2 + l:
        #                    if (i, k, j, l) in two_ints:
        #                        value = two_ints[(i, k, j, l)]
        #                        print(f'{value:23.16e} {i + 1:4d} {j + 1:4d} {k + 1:4d} {l + 1:4d}', file=f)
        
        for p,q in np.array(two_ints.nonzero()).T:
            i,j,k,l = convert_indices(int(np.sqrt(two_ints.shape[0])),int(p), int(q))
            if (i * (i + 1)) / 2 + j >= (k * (k + 1)) / 2 + l:
                print(f'{two_ints[p,q]:23.16e} {i + 1:4d} {j + 1:4d} {k + 1:4d} {l + 1:4d}', file=f)
        
        for p,q in np.array(one_ints.nonzero()).T:
            print(f'{one_ints[p,q]:23.16e} {p + 1:4d} {q + 1:4d} {0:4d} {0:4d}', file=f)

        #core_energy = self.core_energy
        #if core_energy is not None:
        #    print(f'{core_energy:23.16e} {0:4d} {0:4d} {0:4d} {0:4d}', file=f)

In [11]:
filename = "test.txt"
fout = open(filename, "w")
save_fcidump(f=fout, core_energy=ecore, one_ints=h, two_ints=v, nelec=2)

In [6]:
from pyscf import tools
tools.fcidump.read(filename, molpro_orbsym=False)

Parsing test.txt


IndexError: index 9 is out of bounds for axis 0 with size 6

In [12]:
from pyscf import tools
tools.fcidump.read(filename, molpro_orbsym=False)

Parsing test.txt


{'NORB': 2,
 'NELEC': 2,
 'MS2': 0,
 'ORBSYM': [1, 1],
 'ISYM': 1,
 'H1': array([[ 0., -1.],
        [-1.,  0.]]),
 'H2': array([0., 0., 0., 0., 0., 0.])}